In [1]:
import numpy as np
import pandas as pd
import csv
import os
from datetime import datetime

In [2]:
# df = pd.read_csv("csv_files/basketball_2022.csv")
# df.head(5)

In [3]:
def calculate_experience_months(exp):

    start_year = exp
    # Calculate the start and end dates
#     start_date = datetime(int(start_year), 1, 1)
    #change start_date to start counting from 08/01 - when athletes start college??
    start_date = datetime(int(start_year), 8, 1)
    end_date = datetime.now()
#     # Calculate the difference in months
    months_experience = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)
 
    if months_experience < 0:
        months_experience = 0    
    return months_experience

def calculate_experience_years(exp):

    start_year = exp
    # Calculate the start and end dates
    # start_date = datetime(int(start_year), 1, 1)
    # change start_date to start counting from 08/01 - when athletes start college??
    start_date = datetime(int(start_year), 8, 1)
    end_date = datetime.now()
    
    # Calculate the difference in years
    years_experience = (end_date - start_date).days / 365.0
    if years_experience < 0:
        years_experience = 0
    return years_experience
# Apply the function to the 'EXP' column and create a new column 'EXP_MONTHS'


In [4]:
#Looks like most of our features have a large number of missing data
#Some of this is expected, not all athletes have instagram, twitter, or tikok as well as NIL deals
#First we should clean the available data into a more coherent structure
#Within our data there are 2 types of missing values, np.nan and '-', we need to handle both of them and convert them to np.nan
#In our age column, for football 2022 one of the unique values is '1819', this is likely an input error in the On3.com database 

#Future Item notes:
# Remove any atheletes who did not go to college (NBA G League)
# 2 seperate cleaned files supervised and unsupervised

In [5]:
# This function take in a dataframe and cleans all needed columns
def clean_cols(df, sport,rec_year):
    ## FEATURE DICTIONARY
    # NAME - Name of athlete
    # GRADE - College Grade
    #   0: Not Enrolled yet
    #   1: Redshirt Freshmean
    #   2: Freshman
    #   3: Redshirt Sophomore
    #   4: Somphomore
    #   5: Redshirt Junior
    #   6: Junior
    #   7: Redshirt Senior
    #   8: Senior
    # AGE - Age of athlete
    # HOTOWN - Hometown of athlete
    # HISCH - Highschool of athlete
    # STARCOLL - College the athlete has commited to/enrolled at/leading choice
    # NUMOFF - Number of colleges that offered athlete a scholarship
    # POS - Position of athlete
    #   Basketball:
    #       1: PG
    #       2: SG
    #       3: CG
    #       4: SF
    #       5: PF
    #       6: C
    #       7: ATH/Other
    #   Football:
    #       1: QB
    #       2: EDGE
    #       3: WR
    #       4: S
    #       5: DL
    #       6: OT
    #       7: CB
    #       8: LB
    #       9: RB
    #       10: IOL
    #       11: TE
    #       12: K
    #       13: P
    #       14: LS
    #       15: ATH/Other
    # HEIGHT_IN - Athlete height in inches
    # WEIGHT_LBS - Athlete weight in inches
    # SKILL_ - On3.com propriety player skill rating
    # COLLDIST_MI - Distance from athelete hometown from commited/enrolled/top choice college
    # NILVAL_LONG_USD - On3.com NIL valuation
    # INSTA_LONG - # of Instagram followers
    # TWIT_LONG - # of Twitter followers
    # TIK_LONG - # of TikTok followers
    # TOT_FOL - # of social media followers
    # RECRUIT_YEAR - Year the athlete will start their college career
    # EXP_MONTHS - Number of months in college
    # EXP_YEARS - Nummber of years in college
    
    # Clean Position and encode text
    df['POS'] =  [np.nan if (pd.isna(pos) or pos=='-') else pos for pos in df['POS_HEI_WEI'].str.split(' / ').str[0]]
    if sport == 'basketball':
        df['POS'] = [np.nan if pd.isna(pos)
                     else 1 if pos=='PG' else 2 if pos=='SG' else 3 if pos=='CG' else 4 if pos=='SF' else 5 if pos=='PF' else 6 if pos=='C' else 7
                     for pos in df['POS']]
    elif sport == 'football':
        df['POS'] = [np.nan if pd.isna(pos)
                     else 1 if pos=='QB' else 2 if pos=='EDGE' else 3 if pos=='WR' else 4 if pos=='S' else 5 if pos=='DL' else 6 if pos=='OT'
                     else 7 if pos=='CB' else 8 if pos=='LB' else 9 if pos=='RB' else 10 if pos=='IOL' else 11 if pos=='TE' else 12 if pos=='K'
                     else 13 if pos=='P' else 14 if pos=='LS' else 15
                     for pos in df['POS']]
    
    # Clean height and weight
    df['HEIGHT_IN'] = [np.nan if (pd.isna(height) or height=='-') else (float(height.split('-')[0])*12 + float(height.split('-')[1])) for height in df['POS_HEI_WEI'].str.split(' / ').str[1]]
    df['WEIGHT_LBS'] = [np.nan if (pd.isna(weight) or weight=='-') else float(weight) for weight in df['POS_HEI_WEI'].str.split(' / ').str[2]]
    
    # Clean skill rating, remove any + added at the end of a skill rating
    df['SKILL'] = [np.nan if pd.isna(skill) else skill if type(skill)==float else float(skill[0:-1]) if skill[-1]=='+' else float(skill) for skill in df['SKILL']]
    
    # Clean college distance, remove miles
    df['COLLDIST_MI'] = [np.nan if (pd.isna(dist) or dist=='-') else float(dist.split()[0]) for dist in df['COLLDIST']]
    
    # Expand NIL Value and social media from __K or __M format to full number
    df['NILVAL_LONG_USD'] = [np.nan if pd.isna(nilval) else float(nilval[1:-1])*1000000 if nilval[-1]=='M' else float(nilval[1:-1])*1000 if nilval[-1]=='K' else float(nilval[1:-1]) for nilval in df['NILVAL']]
    df['INSTA_LONG'] = [0.0 if (pd.isna(insta) or insta == '-') else float(insta[0:-1])*1000000 if insta[-1]=='M' else float(insta[0:-1])*1000 if insta[-1]=='K' else float(insta) for insta in df['INSTA']]
    df['TWIT_LONG'] = [0.0 if (pd.isna(twit) or twit == '-') else float(twit[0:-1])*1000000 if twit[-1]=='M' else float(twit[0:-1])*1000 if twit[-1]=='K' else float(twit) for twit in df['TWIT']]
    df['TIK_LONG'] = [0.0 if (pd.isna(tik) or tik == '-') else float(tik[0:-1])*1000000 if tik[-1]=='M' else float(tik[0:-1])*1000 if tik[-1]=='K' else float(tik) for tik in df['TIK']]
    df['TOT_FOL'] = df['INSTA_LONG'] = df['TWIT_LONG'] + df['TIK_LONG']
    
    # Find the state of the hometown of the athlete
    df['STATE'] = [np.nan if (pd.isna(ht) or ht=='-') else ht[-3:] for ht in df['HOTOWN']]
    
    # Clean the age and impute missing data, fill with average age of fellow athelete in recruit year
    df['AGE'] = [np.nan if (pd.isna(age) or age=='-') else float(age) for age in df['AGE']]
    df['AGE'] = [df['AGE'].mean() if pd.isna(age) else age for age in df['AGE']]
    if rec_year == '2024':
        df['AGE'] = 17
    elif rec_year == '2025':
        df['AGE'] = 16
    elif rec_year == '2026':
        df['AGE'] = 15

    # Clean grade and encode text
    df['GRADE'] = [np.nan if (pd.isna(grade) or grade=='-') else
                   1 if grade=='Redshirt Freshman' else 2 if grade=='Freshman'
                   else 3 if grade=='Redshirt Sophomore' else 4 if grade=='Sophomore'
                   else 5 if grade=='Redshirt Junior' else 6 if grade=='Junior'
                   else 7 if grade=='Redshirt Senior' else 8 for grade in df['GRADE']]
    if int(rec_year) > 2023:
        df['GRADE'] = 0
    
    # Add sport (for identification purposes)
    df['SPORT'] = sport
    
    # Add year player is/was recruited for
    df['RECRUIT_YEAR'] = int(rec_year)
    
    # Calculate 
    df['EXP_MONTHS'] = df['RECRUIT_YEAR'].apply(calculate_experience_months)
    df['EXP_YEARS'] = df['RECRUIT_YEAR'].apply(calculate_experience_years)
    
    #Remove unneccesary columns from output
    df.drop(['EXP','POS_HEI_WEI','COLLDIST','NILVAL','INSTA','TWIT','TIK','HOTOWN','HISCH','STATE'],axis=1,inplace=True)
    
    return df



In [6]:
# Initialize variables
directory = 'csv_files'
bball_count = 0
fball_count = 0

# Loop to go through each file in the specified directory, run the cleaning function and merge together
for file in os.listdir(directory):
    # Get filepath
    f = os.path.join(directory, file)
    df = pd.read_csv(f)
    
    #Split file name to get the sport and recruit year
    sport = file.split('_')[0]
    year = file.split('_')[-1].split('.')[0]
    
    #Seperate our basketball and football files
    if sport == 'basketball':
        if bball_count == 0:
            bball_clean = clean_cols(df,sport,year)
            bball_count += 1
        else:
            bball_clean = pd.concat([bball_clean,clean_cols(df,sport,year)])
            bball_count += 1
    elif sport == 'football':
        if fball_count == 0:
            fball_clean = clean_cols(df,sport,year)
            fball_count += 1
        else:
            fball_clean = pd.concat([fball_clean,clean_cols(df,sport,year)])
            fball_count += 1
        
print("{} basketball data files cleaned and merged together".format(bball_count))
print("{} footballall data files cleaned and merged together".format(fball_count))


6 basketball data files cleaned and merged together
6 footballall data files cleaned and merged together


In [7]:
# Read in schools dataset
df_schools = pd.read_csv("college_data/schools.csv")

In [8]:
# Split between football and basketball
df_schools_fb = df_schools[df_schools['Sports'].isin(['Football'])]
df_schools_bb = df_schools[df_schools['Sports'].isin(['Basketball'])]

# Remove schools that don't have the shortened university name (will be the merge column)
df_schools_fb.dropna(subset=['institution_name_short'],inplace=True)
df_schools_bb.dropna(subset=['institution_name_short'],inplace=True)

# Simplify dataframe to only the columns we want to merge
df_schools_fb = df_schools_fb[['institution_name_short','ClassificationCode','REV_MEN','EXP_MEN']]
df_schools_bb = df_schools_bb[['institution_name_short','ClassificationCode','REV_MEN','EXP_MEN']]

/var/folders/by/0fsb3r557k7dhr_l1vn7xhxr0000gn/T/ipykernel_12355/461806243.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_schools_fb.dropna(subset=['institution_name_short'],inplace=True)
/var/folders/by/0fsb3r557k7dhr_l1vn7xhxr0000gn/T/ipykernel_12355/461806243.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_schools_bb.dropna(subset=['institution_name_short'],inplace=True)


In [9]:
# Join datasets on institution name
bball_clean_merge = bball_clean.merge(df_schools_bb, left_on='STARCOLL',right_on='institution_name_short',how='outer')
fball_clean_merge = fball_clean.merge(df_schools_fb, left_on='STARCOLL',right_on='institution_name_short',how='outer')


In [10]:
# Write cleaned and merged data to file for unsupervised learning activities
bball_clean_merge.dropna(subset=['GRADE','STARCOLL','WEIGHT_LBS','COLLDIST_MI','AGE','NUMOFF','COLLDIST_MI','SKILL','institution_name_short','HEIGHT_IN'],inplace=True)
fball_clean_merge.dropna(subset=['GRADE','STARCOLL','WEIGHT_LBS','COLLDIST_MI','AGE','NUMOFF','COLLDIST_MI','SKILL','institution_name_short','HEIGHT_IN'],inplace=True)
bball_clean_unsup = bball_clean_merge.drop(['NILVAL_LONG_USD'],axis=1)
fball_clean_unsup = fball_clean_merge.drop(['NILVAL_LONG_USD'],axis=1)
bball_clean_unsup.to_csv('cleaned_files/basketball_clean_unsupervised.csv')
fball_clean_unsup.to_csv('cleaned_files/football_clean_unsupervised.csv')


# Remove athletes that we are missing data for that we can not impute with simple means
bball_clean_sup = bball_clean_merge.dropna(subset=['NILVAL_LONG_USD','GRADE','STARCOLL','WEIGHT_LBS','COLLDIST_MI'])
fball_clean_sup = fball_clean_merge.dropna(subset=['NILVAL_LONG_USD','GRADE','STARCOLL','COLLDIST_MI','WEIGHT_LBS'])

# Write cleaned and merged data to file for supervised learning activities
bball_clean_sup.to_csv('cleaned_files/basketball_clean_supervised.csv')
fball_clean_sup.to_csv('cleaned_files/football_clean_supervised.csv')

,NAME,GRADE,AGE,SKILL,STARCOLL,NUMOFF,POS,HEIGHT_IN,WEIGHT_LBS,COLLDIST_MI,...,TIK_LONG,TOT_FOL,SPORT,RECRUIT_YEAR,EXP_MONTHS,EXP_YEARS,institution_name_short,ClassificationCode,REV_MEN,EXP_MEN
0,Isaiah Collier,2.0,18.000000,99.39,USC,17.0,1.0,75.0,190.0,1921.0,...,0.0,2800.0,basketball,2023.0,2.0,0.172603,USC,1.0,7241439.0,9496433.0
1,Bronny James,2.0,18.000000,97.11,USC,13.0,1.0,75.0,190.0,2045.0,...,5800000.0,5824000.0,basketball,2023.0,2.0,0.172603,USC,1.0,7241439.0,9496433.0
2,Arrinten Page,2.0,18.316667,95.05,USC,20.0,6.0,81.0,225.0,1921.0,...,0.0,761.0,basketball,2023.0,2.0,0.172603,USC,1.0,7241439.0,9496433.0
3,Brandon Gardner,2.0,19.000000,92.20,USC,25.0,5.0,79.0,210.0,2233.0,...,4600.0,7300.0,basketball,2023.0,2.0,0.172603,USC,1.0,7241439.0,9496433.0
5,Vince Iwuchukwu,4.0,19.100000,97.39,USC,11.0,5.0,83.0,220.0,1197.0,...,25000.0,26000.0,basketball,2022.0,14.0,1.172603,USC,1.0,7241439.0,9496433.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242,Kany Tchanda,0.0,17.000000,87.39,Charlotte,9.0,6.0,81.0,210.0,21.0,...,0.0,0.0,basketball,2024.0,0.0,0.000000,Charlotte,1.0,3595247.0,3595247.0
1244,Terry Copeland,0.0,17.000000,86.20,La Salle,2.0,5.0,79.0,225.0,92.0,...,0.0,0.0,basketball,2024.0,0.0,0.000000,La Salle,3.0,3222316.0,3222316.0
1246,A.J. Bates,0.0,17.000000,84.67,Louisiana Tech,6.0,2.0,74.0,180.0,261.0,...,0.0,0.0,basketball,2024.0,0.0,0.000000,Louisiana Tech,1.0,2739125.0,2663394.0
1247,Landren Blocker,0.0,17.000000,83.00,Louisiana Tech,8.0,2.0,76.0,195.0,155.0,...,0.0,0.0,basketball,2024.0,0.0,0.000000,Louisiana Tech,1.0,2739125.0,2663394.0
